In [1]:
import tensorflow as tf
from object_detection.utils import label_map_util
import pathlib
import numpy as np
import cv2
from object_detection.metrics import coco_evaluation
from object_detection.core import standard_fields
import json

2023-12-21 11:19:03.393313: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-21 11:19:03.626719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-21 11:19:03.626762: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-12-21 11:19:03.674244: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 11:19:04.876253: W tensorflow/stream_executor/platform/defaul

Define Paths to TFRECORD file and TFLITE Flatbuffer Models

In [2]:
_PATH_TO_TFLTE_FLATBUFFER = '/home/developer/Documents/git/Tensorflow/workspace/exported_model/tf2/BA/MED3_ssd_resnet_v1_fpn_640x640_3k0/tflite/saved_model/model_quant_metadata.tflite'
_tflite_flatbuffer = pathlib.Path(_PATH_TO_TFLTE_FLATBUFFER)

_PATH_TO_EVAL_TFRECORD = '/home/developer/Documents/git/Tensorflow/workspace/datasets/test.record'
_TFRECORD = pathlib.Path(_PATH_TO_EVAL_TFRECORD)
tfrecord_dataset = tf.data.TFRecordDataset(_TFRECORD)

_PATH_TO_LABELMAP = '/home/developer/Documents/git/Tensorflow/workspace/datasets/dataset/MED3-REV1.00/MED3-REV1.00-detect_label_map.pptxt'
label_map = label_map_util.load_labelmap(_PATH_TO_LABELMAP)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=32, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


2023-12-21 11:19:06.578677: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/developer/Documents/git/Tensorflow/python-env/lib/python3.9/site-packages/cv2/../../lib64:
2023-12-21 11:19:06.578719: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-12-21 11:19:06.578749: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fs-development-machine): /proc/driver/nvidia/version does not exist
2023-12-21 11:19:06.579146: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compi

In [3]:
interpreter = tf.lite.Interpreter(model_path=_PATH_TO_TFLTE_FLATBUFFER)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Check output layer name to determine if this model was created with TF2 or TF1
if ('StatefulPartitionedCall' in output_details[0]['name']):
    # This is a TF2 model
    boxes_idx, classes_idx, scores_idx, count_idx = 1, 3, 0, 2
else:
    # This is a TF1 model
    boxes_idx, classes_idx, scores_idx, count_idx = 0, 1, 2, 3

input_size = interpreter.get_input_details()[0]['shape']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
# load_metadata
displayer = metadata.MetadataDisplayer.with_model_file(_TFLITE_MODEL_WITH_METADATA_PATH)
md_json = json.loads(displayer.get_metadata_json())

In [4]:
def decode_img(encoded_img):
    image_np = tf.image.decode_jpeg(encoded_img, channels=input_size[3]).numpy()
    return image_np

In [5]:
def mobilenet_preprocess_image(encoded_image, input_size):
    image = decode_img(encoded_image)
    image = cv2.resize(image, (input_size[1],input_size[2]))
    image = (image.astype(np.float32) -127.5)/127.5
    image = np.expand_dims(image, axis=0)
    return image

In [13]:
def resnet_preprocess_image(encoded_image, input_size):
    image = decode_img(encoded_image)
    image = cv2.resize(image, (input_size[1],input_size[2]))
    image = np.expand_dims(image, axis=0)
    image = image.astype(np.float32)
    channel_means = [np.float32(123.68), np.float32(116.779), np.float32(103.939)]
    return image - [[channel_means]]

In [ ]:
def preprocess_image(encoded_image, input_size):
    input_type = md_json["subgraph_metadata"][0]["input_tensor_metadata"][0]["description"]
    if input_type == "VGG":
        return resnet_preprocess_image(encoded_image, input_size)
    else:
        return mobilenet_preprocess_image(encoded_image, input_size)

In [7]:
def detect_objects(encoded_image, input_size):
    input_tensor = preprocess_image(encoded_image, input_size)
    interpreter.set_tensor(input_details[0]['index'], input_tensor)
    interpreter.invoke()
    print('num_detections = ', interpreter.get_tensor(output_details[count_idx]['index']) )        
    return {
        'detection_boxes': interpreter.get_tensor(output_details[boxes_idx]['index']),
        'detection_classes': interpreter.get_tensor(output_details[classes_idx]['index']),
        'detection_scores': interpreter.get_tensor(output_details[scores_idx]['index']),
        'num_detections': interpreter.get_tensor(output_details[count_idx]['index'])
    }
    

In [8]:
def get_absolute_bndbox(bbox, height, width):
    ymin, xmin, ymax, xmax = bbox
    ymin = int(ymin * height)
    xmin = int(xmin * width)
    ymax = int(ymax * height)
    xmax = int(xmax * width)
    return [xmin, ymin, xmax, ymax]

In [9]:
def denormalize_groundtruthbox(groundtruthboxes, height, width):
    abs_gtbs = list()
    for i in range(groundtruthboxes.shape[0]):
        abs_gtbs.append(get_absolute_bndbox(groundtruthboxes[i], height , width))

    return np.array(abs_gtbs)

In [14]:
evaluator = coco_evaluation.CocoDetectionEvaluator(categories)
for serialized_example in tfrecord_dataset:
    example = tf.train.Example()
    example.ParseFromString(serialized_example.numpy())
    #run inferenz
    encoded_image = example.features.feature['image/encoded'].bytes_list.value[0]
    image_np = decode_img(encoded_image)
    height = example.features.feature['image/height'].int64_list.value[0]
    width = example.features.feature['image/width'].int64_list.value[0]
    detections = detect_objects(encoded_image, input_size)
    groundtruth_boxes = np.array([example.features.feature['image/object/bbox/ymin'].float_list.value,
                                  example.features.feature['image/object/bbox/xmin'].float_list.value,
                                  example.features.feature['image/object/bbox/ymax'].float_list.value,
                                  example.features.feature['image/object/bbox/xmax'].float_list.value]).T
    
    groundtruth_boxes = denormalize_groundtruthbox(groundtruth_boxes,height,width)
    groundtruth_classes = np.array(example.features.feature['image/object/class/label'].int64_list.value)
    print(groundtruth_classes)
    evaluator.add_single_ground_truth_image_info(
        image_id=example.features.feature['image/source_id'].bytes_list.value[0].decode('utf-8'),
        groundtruth_dict={
            standard_fields.InputDataFields.groundtruth_boxes: groundtruth_boxes,
            standard_fields.InputDataFields.groundtruth_classes: groundtruth_classes,
            standard_fields.InputDataFields.groundtruth_is_crowd: np.array([])
        }
    )

    detection_boxes = denormalize_groundtruthbox(detections['detection_boxes'][0], height, width)
    evaluator.add_single_detected_image_info(
        image_id=example.features.feature['image/source_id'].bytes_list.value[0].decode('utf-8'),
        detections_dict={
            standard_fields.DetectionResultFields.detection_boxes:
            detection_boxes,
            standard_fields.DetectionResultFields.detection_scores:
            np.array(detections['detection_scores'][0]),
            standard_fields.DetectionResultFields.detection_classes:
            np.array(detections['detection_classes'][0].astype(np.int64))+1
        }
    )
    print(detections['detection_classes'][0].astype(np.int64)+1)
evaluator.evaluate()


num_detections =  [25.]
[ 1  3  5  8 11 14 17 20 23 26 29 31]
[ 3 29 26  1 23 14 20  5  8 17 11 31 20 11 31 17  8  3  8 20 26 17 29  5
 14]
num_detections =  [25.]
[ 1  3  5  8 11 14 17 20 23 26 29 31]
[ 3 29 26 23  8 20 14  1  5 17 31 11  8 31 17 26  1  8 31 20  3 11 14 14
 17]
num_detections =  [25.]
[ 1  3  5  8 11 14 17 20 23 26 29 31]
[ 3 29 26  1 23 20 14  8  5 17 11 31  3 14  8 20 20  5 11  8  8 26 31 26
 29]
num_detections =  [25.]
[ 1  3  5  8 11 14 17 20 23 26 29 31]
[ 3 29 26  1 23 14  8 20  5 11 17 31 17 20 11 31 26  8  8 29 17 11  1 17
 26]
num_detections =  [25.]
[ 1  3  5  8 11 14 17 20 23 26 29 31]
[ 3 29 26 23  1  8 14 20  5 17 31 11  8  8 31 11 31 20 17  8  5 26 26  1
 17]
num_detections =  [25.]
[ 1  3  5  8 11 14 17 20 23 26 29 31]
[ 3 29 26  1 23  8 17 20  5 14 31 11 17 20 31 23 11  8 20 31 26 29  3 20
 11]
num_detections =  [25.]
[ 1  3  5  8 11 14 17 20 23 26 29 31]
[ 3 29 26  1  8 23 20 14 31 17 11  5  8 31 11 20 31 20  8  1 17 31 11 26
  5]
num_detections =  [2

{'DetectionBoxes_Precision/mAP': 0.7107729568770493,
 'DetectionBoxes_Precision/mAP@.50IOU': 0.9894460329723678,
 'DetectionBoxes_Precision/mAP@.75IOU': 0.8601758710423834,
 'DetectionBoxes_Precision/mAP (small)': 0.6291940586393155,
 'DetectionBoxes_Precision/mAP (medium)': 0.7045796278621995,
 'DetectionBoxes_Precision/mAP (large)': 0.877906596875209,
 'DetectionBoxes_Recall/AR@1': 0.7616503124251026,
 'DetectionBoxes_Recall/AR@10': 0.762243404229132,
 'DetectionBoxes_Recall/AR@100': 0.762243404229132,
 'DetectionBoxes_Recall/AR@100 (small)': 0.6656593173720562,
 'DetectionBoxes_Recall/AR@100 (medium)': 0.7537024334755295,
 'DetectionBoxes_Recall/AR@100 (large)': 0.8986556036556036}